In [6]:
# First, completely clean all Qiskit installations
!pip uninstall -y qiskit qiskit-terra qiskit-aer qiskit-ibmq-provider qiskit-optimization qiskit-algorithms --quiet
!pip uninstall -y qiskit-nature qiskit-finance qiskit-machine-learning --quiet  # Remove other Qiskit packages

# Install EXACT versions that work together (Qiskit 1.0 ecosystem)
!pip install numpy==1.26.4
!pip install scipy==1.13.1
!pip install pylatexenc==2.10

# Core Qiskit packages
!pip install qiskit==1.4.2
!pip install qiskit_algorithms==0.3.1
!pip install qiskit_optimization==0.6.1
!pip install qiskit_nature==0.7.2
#"qiskit_optimization>=0.6.0" "qiskit_algorithms>=0.3.0"
!pip install qiskit-aer>==0.15.1

# For Quantinuum hardware access
!pip install pytket==2.1.0
!pip install pytket-qiskit==0.66.0
!pip install pytket-quantinuum==0.46.0
!pip install qnexus>=1.6.0


  Using cached qiskit-1.4.2-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
Using cached qiskit-1.4.2-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (6.8 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytket-qiskit 0.66.0 requires qiskit-aer>=0.15.1, which is not installed.
  Using cached qiskit_algorithms-0.3.1-py3-none-any.whl.metadata (4.2 kB)
Using cached qiskit_algorithms-0.3.1-py3-none-any.whl (310 kB)
  Using cached qiskit_optimization-0.6.1-py3-none-any.whl.metadata (8.6 kB)
Using cached qiskit_optimization-0.6.1-py3-none-any.whl (167 kB)
  Using cached qiskit_nature-0.7.2-py3-none-any.whl.metadata (8.0 kB)
Using cached qiskit_nature-0.7.2-py3-none-any.whl (2.2 MB)
  Using cached pytket-2.1.0-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (4.7 kB)
Using cached pytket-2.1.0-cp311-cp311-manyli

In [2]:
import numpy as np
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit.primitives import Sampler
from qiskit_algorithms import QAOA
from qiskit.algorithms.optimizers import COBYLA
from qiskit import QuantumCircuit
from pytket.extensions.qiskit import qiskit_to_tk
import datetime
from pandas import DataFrame
import qnexus as qnx

ImportError: cannot import name 'Sampler' from 'qiskit.primitives' (/usr/local/lib/python3.11/dist-packages/qiskit/primitives/__init__.py)

In [ ]:
qnx.login_with_credentials() # sign in via python prompt


In [ ]:
project=qnx.projects.get_or_create(name="YQuantum LTIMindTree/Travelers Project")
qnx.context.set_active_project(project)

quCMP1 = "H1-1LE"  # noiseless quantum computer
quCMP2 = "H1-Emulator" # full experience quantum computer (extra noise)
config = qnx.QuantinuumConfig(device_name=quCMP1) # Choose the quantum computer to use

In [ ]:
# Create quantum circuit
jobname_suffix = datetime.datetime.now().strftime("%Y_%m_%d-%H-%M-%S")  # use to create unique jobIDs
n_qubits = 20
qp = QuadraticProgram()
for i in range(20):
    qp.binary_var(f"x{i}")
# EXAMPLE: minimize x0 + x1 + x0*x1
qp.minimize(
    linear={f"x0": 1, "x1": 1},
    quadratic={("x0", "x1"): 2}
)

qubo = QuadraticProgramToQubo().convert(qp)
operator, offset = qubo.to_ising()

qaoa = QAOA(sampler=Sampler(),COBYLA(maxiter=100), reps=1)
qaoa_circuit = qaoa.construct_circuit(operator)
qc = qaoa_circuit

qc.measure_all()
tket_circuit = qiskit_to_tk(qc)
ref = qnx.circuits.upload(circuit=tket_circuit, name=f"GHZ-Circuit-{jobname_suffix}")
render_circuit_jupyter(ref.download_circuit())

# Compile quantum circuit
ref_compile_job = qnx.start_compile_job(
    circuits=[ref],
    backend_config=config,
    optimisation_level=2,
    name=f"compilation-job-{jobname_suffix}",
)
qnx.jobs.wait_for(ref_compile_job)
ref_compiled_circuit = qnx.jobs.results(ref_compile_job)[0].get_output()
compiled_circuit = ref_compiled_circuit.download_circuit()


In [ ]:
# Execute with nexus
n_shots = 10
ref_execute_job = qnx.start_execute_job(
    circuits=[ref_compiled_circuit],
    backend_config=config,
    n_shots=[n_shots],
    name=f"execute-job-{jobname_suffix}"
)

qnx.jobs.wait_for(ref_execute_job)
ref_result = qnx.jobs.results(ref_execute_job)[0]
result = ref_result.download_result()

distribution = result.get_distribution()
df = DataFrame(distribution.values(), index=list(distribution.keys()), columns=["Probability"])
df